In [ ]:
import networkx as nx
%matplotlib inline

# Chapter 2 Tutorial

多くの練習問題の後に `assert` 文を含むブロックがあることに注意されたい。この `assert` 文の前にセットアップコードがある場合がある。これはあなたが正しい道にいるというフィードバックを与えるために設置されている。もし `AssertionError` を受け取ったら、おそらくどこかに間違いがあるということだ。

Contents:

1. 経路（Paths）
2. 連結成分（Connected components）
3. 有向路（Directed paths & components）
4. Dataset: US air traffic network

## 1. 経路（Path）

無向ネットワークという簡単な例から始めよう。

In [ ]:
G = nx.Graph()

G.add_nodes_from([1,2,3,4])

G.add_edges_from([(1,2),(2,3),(1,3),(1,4)])

nx.draw(G, with_labels=True)

ネットワーク内の経路（*path*）とは、二つのノードをつなぐエッジの集合（エッジに沿ってたどることができる道筋）である。 この単純な例では、ノード3と4を結ぶパスが少なくとも1つ存在することが簡単にわかる。このことは NetworkX で確認できる。

In [ ]:
nx.has_path(G, 3, 4)

二つのノードの間に複数の経路がある場合がある。再びノード3と4を考えたとき、そのような「単純な」経路が二つある。

In [ ]:
list(nx.all_simple_paths(G, 3, 4))

単純な経路（*simple path*）とは、サイクルのない経路である。もしサイクルを許せば、いつでも好きなだけサイクルを回れるので、無限の経路が存在することになる。

私たちはしばしば、最短経路（*shortest path*）に最も興味を持つ。重みのないネットワークでは、最短経路は最も少ないエッジを持つ経路である。ノード3とノード4の間の2つの単純な経路のうち、一方が他方より短いことがわかる。この最短経路は、NetworkX の関数を使うことで得ることができる。

In [ ]:
nx.shortest_path(G, 3, 4)

経路長（*path length*）にのみ関心がある時は、次の関数が用意されている。

In [ ]:
nx.shortest_path_length(G, 3, 4)

ここで、経路長が経路に含まれるノードの数ではなく、**エッジ**の数として定義されていることに注意しよう。つまり、ノード $u$ と $v$ に対して、

    nx.shortest_path_length(G, u, v) == len(nx.shortest_path(G, u, v)) - 1



## 2. 連結成分（*connected components*）

上の単純なネットワークでは、どのノードのペアに対してもそれらをつなぐ経路が存在する。これは連結グラフ（*connected graph*）の定義である。所与のグラフについて、次のようにしてこの連結性を確認することができる。

In [ ]:
nx.is_connected(G)

すべてのグラフが連結しているわけではない。

In [ ]:
G = nx.Graph()

G.add_cycle((1,2,3))
G.add_edge(4,5)

nx.draw(G, with_labels=True)

In [ ]:
nx.is_connected(G)

ノード間に経路が存在しないのに経路を確かめようとすると、NetworkX はエラーを返してくる。

In [ ]:
nx.shortest_path(G ,3, 5)

このグラフにおいて、二つの連結成分を視覚的に確認できるが、コードで検証してみよう。

In [ ]:
nx.number_connected_components(G)

関数 `nx.connected_components()` はグラフを受け取り、それぞれの連結成分に含まれるノードの集合を要素に持つリストを返す。次のリストに格納された二つの集合が、上で描画したグラフにある二つの連結成分に対応していることを確認しよう。

In [ ]:
list(nx.connected_components(g))

Pythonの集合に馴染みがない方のために説明すると、集合とは重複のない要素のコレクションのことである。ノード名は一意なので、ノード名を収集するのに便利なデータ型だ。リストのような他のコレクションと同様に、 `len` 関数で集合内の要素数を取得できる。

In [ ]:
components = list(nx.connected_components(G))
len(components[0])

私たちはしばしば、ネットワークのコア（*core*）とも呼ばれる最大連結成分に注目する。最大連結成分を得るには、Pythonの組み込みの `max()` 関数を使用することができる。デフォルトでは、Pythonの `max()` 関数は辞書順（すなわちアルファベット順）でソートするが、これはここでは役に立たない。ここでは、大きさの順にソートされたときの最大連結成分が欲しいので、キー関数に `len` を渡す。

In [ ]:
max(nx.connected_components(G), key=len)

ノード名のリストを得るだけで十分な場合もあるが、最大連結成分からなる実際のサブグラフが必要な場合もある。これを得るための1つの方法は、ノード名のリストを `G.subgraph()` 関数に渡すことです。

In [ ]:
core_nodes = max(nx.connected_components(G), key=len)
core = G.subgraph(core_nodes)

nx.draw(core, with_labels=True)

タブのコード補完機能を使用している人は、`nx.connected_component_subgraphs()`関数にも気が付くはずだ。これもコアサブグラフを得るために使用できるが、最大連結成分にしか興味がない場合は、先に紹介した方法の方が効率的である。

## 3. 有向路と有向成分（Directed paths & components）

これまで見てきた経路と連結成分のアイデアを、有向グラフにも拡張してみよう。

In [ ]:
D = nx.Digraph()
D.add_edges_from([
    (1,2),
    (2,3),
    (3,2), (3,4), (3,5),
    (4,2), (4,5), (4,6),
    (5,6),
    (6,4),
])
nx.draw(D, with_labels=True)

### 有向路（directed paths）

有向グラフにおいて、任意のノード$u$から任意のノード$v$に向かうエッジは、$v$から$u$へのエッジの存在を意味しない。有向グラフでは、経路はエッジの方向に従わなければならないので、経路についても同じ非対称性がある。このグラフには、1から4への経路はあるが、逆方向の経路はないことを確認せよ。

In [ ]:
nx.has_path(D, 1, 4)

In [ ]:
nx.has_path(D, 4, 1)

経路を扱うNetworkX のもう一つの関数も同様にこの非対称性を考慮している。

In [ ]:
nx.shortest_path(D, 2, 5)

In [ ]:
nx.shortest_path(D, 5, 2)

5から3に向かうエッジはないので5から2に向かう最短経路は、2から5に向かう最短経路を逆方向にただ辿っていくことはできない。ノード6と4を経由する迂回路を通るしかない。

### 有向成分（directed components）

有向ネットワークには、2種類の連結性がある。強連結（*strongly connected*）とは、すべてのノードのペアの間に有向路が存在すること、つまり、どのノードからでもエッジの方向性に従いながら他のノードに到達できることを意味する。一方通行の道路網を走る車を思い浮かべてほしい。車の流れに逆らって運転することはできない。

In [ ]:
nx.is_strongly_connected(D)

弱連結（*weakly connected*）とは、方向に関係なく、すべてのノードのペアの間に経路が存在することを意味する。一方通行の道路網にいる歩行者を考えてみよう。彼らは歩道を歩くので、交通の方向を気にすることはない。

In [ ]:
nx.is_weakly_connected(D)

もしネットワークが強連結しているなら、弱連結も満たしていることになる。逆はこの例に見られるように、必ずしも真ではない。

無向グラフのための `is_connected()` 関数は、有向グラフが与えられたときエラーを返すようになっている。

In [ ]:
# これはエラーを返す
nx.is_connected(D)

有向グラフの場合、`nx.connected_components()`の代わりに `nx.weakly_connected_components()` と `nx.strongly_connected_components()` を使うことになる。

In [ ]:
list(nx.weakly_connected_components(D))

In [ ]:
list(nx.strongly_connected_components(D))

## 4. データセット：US air traffic network

このリポジトリには、いくつかのネットワークデータセットが含まれている。その中には、アメリカの航空路線のネットワークがある。

In [ ]:
G = nx.read_graphml('../datasets/openflights/openflights_usa.graphml.gz')

このグラフのノードは空港で、[IATAコード](https://en.wikipedia.org/wiki/List_of_airports_by_IATA_code:_A)で表されている。2つのノードは、2つの空港を直接結ぶ定期便がある場合、エッジで結ばれる。一方向のフライトは通常、帰りのフライトがあることを意味するので、このグラフは無向グラフと仮定する。

このグラフはエッジ
```
[('HOM', 'ANC'), ('BGM', 'PHL'), ('BGM', 'IAD'), ...]
```
を持ち、ANC はアンカレッジ、IAD はワシントン・ダレス etc. を表す。

これらのノードはまた、自身に関連する属性（*attributes*）を持っており、空港についての追加情報が保存されている。

In [ ]:
G.nodes['IND']

ノードの属性は辞書形式で保存されており、次のようにして個々の値にアクセスできる。

In [ ]:
G.nodes['IND']['name']

## 練習問題1

インディアナポリス（Indianapolis）とアラスカ州フェアバンクス(FAI)の間に直行便はあるか？直行便とは、途中で降機しない便のことである。

In [ ]:
nx.shortest_path(G, 'IND', 'FAI')

## 練習問題2

今仮に、インディアナポリス（Indianapolis）からアラスカ州フェアバンクス(FAI)に飛びたいとする。フライトの数を最小にする旅行プランはどんなものになるだろうか？

## 練習問題3

米国内のどの空港からでも、場合によっては乗り継ぎ便を利用して、米国内の他のどの空港へも移動することは可能だろうか？つまり、航空路線ネットワークには、考えられるすべての空港のペアを結ぶ経路が存在するだろうか？

## 解答例

### 練習問題1

In [ ]:
nx.shortest_path(G, 'IND', 'FAI')

INDからFAIに移動する最短経路が `['IND', 'FAI']` ではないので、2点を結ぶ直行便はない。

### 練習問題2

INDとFAIを結ぶ最短経路が `['IND', 'BOS', 'SEA', 'FAI']` なので、フライト数を最小にする旅程は、`IND -> BOS -> SEA -> FAI` になる。

### 練習問題3

In [ ]:
nx.is_connected(G)

よって、どの空港からでも任意の空港に到達できるとはいえない。このネットワークは以下のように、3つの連結成分からなっている。

In [ ]:
len(list(nx.connected_components(G)))

In [ ]:
list(nx.connected_components(G))